In [1]:
import pandas as pd 
import numpy as np 

# Import matplotlib and make sure it outputs plots
%matplotlib inline 
import matplotlib.pyplot as plt

# Import seaborn and set the style
import seaborn as sns
sns.set(color_codes=True)
sns.set_style('darkgrid')

In [2]:
pwc = pd.ExcelFile("Ananlytics Case Study Dataset_412019.xlsx")
claims = pd.read_excel(pwc, 'Claims_Fact')
risk = pd.read_excel(pwc, 'Risk_Fact')
driver = pd.read_excel(pwc, 'Driver_Dimension')
policy = pd.read_excel(pwc, 'Policy_Dimension')
vehicles = pd.read_excel(pwc, 'Vehicles_Dimension')
pop = pd.read_excel(pwc, 'Population Distribution')
location = pd.read_excel(pwc, 'Location_Dimension')
claimexp = pd.read_excel(pwc, 'Claim Expense(LAE)_Dimension')

In [3]:
claims.rename(columns={'Claimant Id':'Driver ID'}, inplace=True)
claims.head(5)

,Claim ID,Policy ID,Driver ID,Vehicle ID,Claim Amount,Claim Description
0,1016,1015072,48946,86,112800,Loss of Value to Vehicle
1,319,1016048,2218184,84,108580,Loss of Value to Vehicle
2,162,1017411,20411,85,106720,Loss of Value to Vehicle
3,1014,1027330,30330,87,104580,Injury by Uninsured or Underinsured Defendant
4,352,1000458,3458,73,6580,"Damage from impacts with animals (deer, etc.);..."


In [4]:
#Add a Age column in the driver dataframe
driver['Date of Birth'] = pd.to_datetime(driver['Date of Birth'], errors='coerce')
driver['Year'] = driver['Date of Birth'].dt.year
driver['Age'] = 2015 - driver['Year']
driver.head(5)

,Driver ID,First Name,Last Name,Number of Violations,Number of Accidents,Marital Status,Gender,Miles to work,Primary Vehicle Use,Date of Birth,Year,Age
0,1,Duane,Crance,3,0,M,M,9.0,Work,1952-04-16,1952.0,63.0
1,2,Sheldon,Sherbert,1,0,S,M,8.0,Leisure,1971-11-15,1971.0,44.0
2,3,Kristian,Brix,0,0,M,M,40.0,Work,1989-01-13,1989.0,26.0
3,4,Renea,Kreck,4,2,S,M,4.0,Work,1990-06-07,1990.0,25.0
4,5,Easter,Mccrossen,0,0,S,F,19.0,Work,1999-05-02,1999.0,16.0


In [5]:
#Create dummy variable Gender2 and PrimUse
driver['Gender2'] = np.where(driver['Gender'] == 'M',1 , 0).copy()
driver['PrimUse'] = np.where(driver['Primary Vehicle Use'] == 'Work',1 , 0).copy()
driver.head(5)

,Driver ID,First Name,Last Name,Number of Violations,Number of Accidents,Marital Status,Gender,Miles to work,Primary Vehicle Use,Date of Birth,Year,Age,Gender2,PrimUse
0,1,Duane,Crance,3,0,M,M,9.0,Work,1952-04-16,1952.0,63.0,1,1
1,2,Sheldon,Sherbert,1,0,S,M,8.0,Leisure,1971-11-15,1971.0,44.0,1,0
2,3,Kristian,Brix,0,0,M,M,40.0,Work,1989-01-13,1989.0,26.0,1,1
3,4,Renea,Kreck,4,2,S,M,4.0,Work,1990-06-07,1990.0,25.0,1,1
4,5,Easter,Mccrossen,0,0,S,F,19.0,Work,1999-05-02,1999.0,16.0,0,1


Merged dataframe 'dcr' = Driver Dimension, Claims Fact and Risk Fact

In [6]:
dc = pd.merge(left=claims, right=driver, how='left', on=['Driver ID'])
dc.head(5)

,Claim ID,Policy ID,Driver ID,Vehicle ID,Claim Amount,Claim Description,First Name,Last Name,Number of Violations,Number of Accidents,Marital Status,Gender,Miles to work,Primary Vehicle Use,Date of Birth,Year,Age,Gender2,PrimUse
0,1016,1015072,48946,86,112800,Loss of Value to Vehicle,Noelia,Meleski,2,0,M,M,23.0,Leisure,1978-11-24,1978.0,37.0,1,0
1,319,1016048,2218184,84,108580,Loss of Value to Vehicle,Karan,Bogdon,1,0,S,M,40.0,Work,1985-09-18,1985.0,30.0,1,1
2,162,1017411,20411,85,106720,Loss of Value to Vehicle,Kyung,Jansky,1,0,D,M,15.0,Work,1972-12-31,1972.0,43.0,1,1
3,1014,1027330,30330,87,104580,Injury by Uninsured or Underinsured Defendant,Jana,Holroyd,6,4,D,M,6.0,Leisure,1990-06-12,1990.0,25.0,1,0
4,352,1000458,3458,73,6580,"Damage from impacts with animals (deer, etc.);...",Mayme,Goedecke,1,1,D,M,10.0,Leisure,1994-11-15,1994.0,21.0,1,0


In [7]:
dc.shape

(1354, 19)

In [8]:
risk2 = pd.pivot_table(risk[['Driver ID', 'Vehicle ID', 'Vehicle Model Year', 'Location ID', 'Total Premium']],index=['Driver ID', 'Vehicle ID', 'Vehicle Model Year', 'Location ID'], values =['Total Premium'], aggfunc=np.sum)
risk2 = risk2.reset_index()

In [9]:
dcr = pd.merge(left=dc, right=risk2, how='left', on=['Driver ID'])
dcr.head(5)

,Claim ID,Policy ID,Driver ID,Vehicle ID_x,Claim Amount,Claim Description,First Name,Last Name,Number of Violations,Number of Accidents,...,Primary Vehicle Use,Date of Birth,Year,Age,Gender2,PrimUse,Vehicle ID_y,Vehicle Model Year,Location ID,Total Premium
0,1016,1015072,48946,86,112800,Loss of Value to Vehicle,Noelia,Meleski,2,0,...,Leisure,1978-11-24,1978.0,37.0,1,0,86,2006,236,2075.929804
1,319,1016048,2218184,84,108580,Loss of Value to Vehicle,Karan,Bogdon,1,0,...,Work,1985-09-18,1985.0,30.0,1,1,84,2012,431,2694.570900
2,162,1017411,20411,85,106720,Loss of Value to Vehicle,Kyung,Jansky,1,0,...,Work,1972-12-31,1972.0,43.0,1,1,53,2008,451,1095.710626
3,1014,1027330,30330,87,104580,Injury by Uninsured or Underinsured Defendant,Jana,Holroyd,6,4,...,Leisure,1990-06-12,1990.0,25.0,1,0,54,2008,590,6498.570900
4,352,1000458,3458,73,6580,"Damage from impacts with animals (deer, etc.);...",Mayme,Goedecke,1,1,...,Leisure,1994-11-15,1994.0,21.0,1,0,73,2009,266,2694.570900


In [10]:
dcr.shape

(1354, 23)

Merged dataframe 'drl' = Driver Dimension, Risk Fact and Location Dimension

In [11]:
dr = pd.merge(left=risk, right=driver, how='left', on=['Driver ID'])
dr.head(5)

,Record ID,Policy Number,Driver ID,Vehicle ID,Vehicle Model Year,Location ID,Total Premium,First Name,Last Name,Number of Violations,Number of Accidents,Marital Status,Gender,Miles to work,Primary Vehicle Use,Date of Birth,Year,Age,Gender2,PrimUse
0,1,1001758,4758,50,2006,65,3450.5709,Colton,Bowser,4,0,M,M,20.0,Work,1953-07-03,1953.0,62.0,1,1
1,2,1002901,5901,68,2013,536,1938.5709,Venita,Streva,0,0,S,M,2.0,Leisure,1970-06-18,1970.0,45.0,1,0
2,3,1009972,12972,43,2005,,3174.5709,Numbers,Madise,1,0,M,M,9.0,Leisure,1993-09-03,1993.0,22.0,1,0
3,4,1014384,17384,7,2005,228,2958.5709,Shelba,Poirrier,1,1,M,M,2.0,Work,1992-07-09,1992.0,23.0,1,1
4,5,1014425,17425,73,2013,364,762.5709,Nerissa,Leckband,0,0,D,M,4.0,Work,1958-03-16,1958.0,57.0,1,1


In [12]:
dr.shape

(39069, 20)

In [13]:
#Change the blank value in driver_risk['Location ID'] to -1
dr.loc[dr['Location ID']==' ', 'Location ID'] = -1

In [14]:
#Change the data type of driver_risk['Location ID'] from object to int64
dr['Location ID'] = dr['Location ID'].astype('int64')

In [15]:
drl = pd.merge(left=dr, right=location, how='left', on=['Location ID'])
drl.head(5)

,Record ID,Policy Number,Driver ID,Vehicle ID,Vehicle Model Year,Location ID,Total Premium,First Name,Last Name,Number of Violations,...,Date of Birth,Year,Age,Gender2,PrimUse,ZIP,Primary_City,State,County,Estimated_population_2013
0,1,1001758,4758,50,2006,65,3450.5709,Colton,Bowser,4,...,1953-07-03,1953.0,62.0,1,1,92630.0,Lake Forest,CA,Orange County,56565.0
1,2,1002901,5901,68,2013,536,1938.5709,Venita,Streva,0,...,1970-06-18,1970.0,45.0,1,0,90222.0,Compton,CA,Los Angeles County,26204.0
2,3,1009972,12972,43,2005,-1,3174.5709,Numbers,Madise,1,...,1993-09-03,1993.0,22.0,1,0,NaN,NaN,NaN,NaN,NaN
3,4,1014384,17384,7,2005,228,2958.5709,Shelba,Poirrier,1,...,1992-07-09,1992.0,23.0,1,1,92880.0,Corona,CA,Riverside County,40234.0
4,5,1014425,17425,73,2013,364,762.5709,Nerissa,Leckband,0,...,1958-03-16,1958.0,57.0,1,1,91780.0,Temple City,CA,Los Angeles County,33012.0


Merged dataframe 'dcrcl' = Driver Dimension, Claims Fact, Claims Expense, Location Dimension and Risk Fact

In [16]:
dcrc = pd.merge(left=dcr, right=claimexp, how='left', on=['Claim ID'])
dcrc.head(5)

,Claim ID,Policy ID,Driver ID,Vehicle ID_x,Claim Amount,Claim Description,First Name,Last Name,Number of Violations,Number of Accidents,...,Year,Age,Gender2,PrimUse,Vehicle ID_y,Vehicle Model Year,Location ID,Total Premium,Expense Amount,Claim Handling Branch
0,1016,1015072,48946,86,112800,Loss of Value to Vehicle,Noelia,Meleski,2,0,...,1978.0,37.0,1,0,86,2006,236,2075.929804,19176.0,1
1,319,1016048,2218184,84,108580,Loss of Value to Vehicle,Karan,Bogdon,1,0,...,1985.0,30.0,1,1,84,2012,431,2694.570900,18458.6,3
2,162,1017411,20411,85,106720,Loss of Value to Vehicle,Kyung,Jansky,1,0,...,1972.0,43.0,1,1,53,2008,451,1095.710626,1067.2,2
3,1014,1027330,30330,87,104580,Injury by Uninsured or Underinsured Defendant,Jana,Holroyd,6,4,...,1990.0,25.0,1,0,54,2008,590,6498.570900,10458.0,4
4,352,1000458,3458,73,6580,"Damage from impacts with animals (deer, etc.);...",Mayme,Goedecke,1,1,...,1994.0,21.0,1,0,73,2009,266,2694.570900,1052.8,1


In [17]:
#Change the blank value in driver_risk['Location ID'] to -1
dcrc.loc[dcrc['Location ID']==' ', 'Location ID'] = -1

In [18]:
#Change the data type of driver_risk['Location ID'] from object to int64
dcrc['Location ID'] = dcrc['Location ID'].astype('int64')

In [19]:
dcrcl = pd.merge(left=dcrc, right=location, how='left', on=['Location ID'])
dcrcl.head(5)

,Claim ID,Policy ID,Driver ID,Vehicle ID_x,Claim Amount,Claim Description,First Name,Last Name,Number of Violations,Number of Accidents,...,Vehicle Model Year,Location ID,Total Premium,Expense Amount,Claim Handling Branch,ZIP,Primary_City,State,County,Estimated_population_2013
0,1016,1015072,48946,86,112800,Loss of Value to Vehicle,Noelia,Meleski,2,0,...,2006,236,2075.929804,19176.0,1,90006,Los Angeles,CA,Los Angeles County,39848
1,319,1016048,2218184,84,108580,Loss of Value to Vehicle,Karan,Bogdon,1,0,...,2012,431,2694.570900,18458.6,3,91340,San Fernando,CA,Los Angeles County,30191
2,162,1017411,20411,85,106720,Loss of Value to Vehicle,Kyung,Jansky,1,0,...,2008,451,1095.710626,1067.2,2,94561,Oakley,CA,Contra Costa County,29435
3,1014,1027330,30330,87,104580,Injury by Uninsured or Underinsured Defendant,Jana,Holroyd,6,4,...,2008,590,6498.570900,10458.0,4,95117,San Jose,CA,Santa Clara County,24208
4,352,1000458,3458,73,6580,"Damage from impacts with animals (deer, etc.);...",Mayme,Goedecke,1,1,...,2009,266,2694.570900,1052.8,1,94531,Antioch,CA,Contra Costa County,38379


In [20]:
dcrcl.shape

(1354, 30)

### Regression Analysis

In [21]:
import statsmodels.formula.api as smf

In [22]:
drl.rename(columns={'Total Premium':'Total_Premium'}, inplace=True)
drl.rename(columns={'Number of Violations':'Number_of_Violations'}, inplace=True)

In [23]:
#First regression
results1 = smf.ols('Total_Premium ~ Age + Number_of_Violations + Estimated_population_2013 + C(Gender2) + C(PrimUse)',
                  data=drl).fit()
results1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Total_Premium   R-squared:                       0.049
Model:                            OLS   Adj. R-squared:                  0.049
Method:                 Least Squares   F-statistic:                     384.5
Date:                Thu, 18 Apr 2019   Prob (F-statistic):               0.00
Time:                        17:06:12   Log-Likelihood:            -3.1687e+05
No. Observations:               37121   AIC:                         6.337e+05
Df Residuals:                   37115   BIC:                         6.338e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                  1586.0534     34.440     46.053      0.000    1518.551    1653.556
C(Gender2)[T.1]              -0.6838     13.295     -0.051      0.959     -26.742      25.374
C(PrimUse)[T.1]              -6.1628     14.772     -0.417      0.677     -35.116      22.790
Age                          -1.8632      0.503     -3.704      0.000      -2.849      -0.877
Number_of_Violations        151.9751      3.547     42.848      0.000     145.023     158.927
Estimated_population_2013     0.0003      0.000      0.709      0.479      -0.001       0.001
==============================================================================
Omnibus:                     1401.476   Durbin-Watson:                   1.040
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1825.011
Skew:                          -0.410   Prob(JB):                         0.00
Kurtosis:                       3.713   Cond. No.                     2.36e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.36e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [29]:
dcrcl.rename(columns={'Expense Amount':'Expense_Amount'}, inplace=True)
dcrcl.rename(columns={'Number of Violations':'Number_of_Violations'}, inplace=True)

In [25]:
#Second regression
results2 = smf.ols('Expense_Amount ~ Age + Estimated_population_2013 + C(Gender2) + C(PrimUse)',
                  data=dcrcl).fit()
results2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         Expense_Amount   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.2505
Date:                Thu, 18 Apr 2019   Prob (F-statistic):              0.909
Time:                        17:06:12   Log-Likelihood:                -11401.
No. Observations:                1292   AIC:                         2.281e+04
Df Residuals:                    1287   BIC:                         2.284e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                  1029.1050    236.133      4.358      0.000     565.856    1492.354
C(Gender2)[T.1]              39.5944     96.273      0.411      0.681    -149.275     228.464
C(PrimUse)[T.1]             -91.6586    102.732     -0.892      0.372    -293.200     109.882
Age                           0.5316      3.548      0.150      0.881      -6.429       7.492
Estimated_population_2013     0.0009      0.004      0.256      0.798      -0.006       0.008
==============================================================================
Omnibus:                     1276.665   Durbin-Watson:                   1.119
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            61718.652
Skew:                           4.688   Prob(JB):                         0.00
Kurtosis:                      35.535   Cond. No.                     2.24e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.24e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [30]:
dcrcl['Loss_Ratio'] = (dcrcl['Claim Amount'] + dcrcl['Expense_Amount']) / dcrcl['Total Premium']
dcrcl.head(5)

,Claim ID,Policy ID,Driver ID,Vehicle ID_x,Claim Amount,Claim Description,First Name,Last Name,Number_of_Violations,Number of Accidents,...,Location ID,Total Premium,Expense_Amount,Claim Handling Branch,ZIP,Primary_City,State,County,Estimated_population_2013,Loss_Ratio
0,1016,1015072,48946,86,112800,Loss of Value to Vehicle,Noelia,Meleski,2,0,...,236,2075.929804,19176.0,1,90006,Los Angeles,CA,Los Angeles County,39848,63.574404
1,319,1016048,2218184,84,108580,Loss of Value to Vehicle,Karan,Bogdon,1,0,...,431,2694.570900,18458.6,3,91340,San Fernando,CA,Los Angeles County,30191,47.146134
2,162,1017411,20411,85,106720,Loss of Value to Vehicle,Kyung,Jansky,1,0,...,451,1095.710626,1067.2,2,94561,Oakley,CA,Contra Costa County,29435,98.371958
3,1014,1027330,30330,87,104580,Injury by Uninsured or Underinsured Defendant,Jana,Holroyd,6,4,...,590,6498.570900,10458.0,4,95117,San Jose,CA,Santa Clara County,24208,17.702046
4,352,1000458,3458,73,6580,"Damage from impacts with animals (deer, etc.);...",Mayme,Goedecke,1,1,...,266,2694.570900,1052.8,1,94531,Antioch,CA,Contra Costa County,38379,2.832659


In [31]:
#Third regression
results3 = smf.ols('Loss_Ratio ~ Age + Number_of_Violations + Estimated_population_2013 + C(Gender2) + C(PrimUse)',
                  data=dcrcl).fit()
results3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Loss_Ratio   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.153
Date:                Thu, 18 Apr 2019   Prob (F-statistic):              0.331
Time:                        17:15:08   Log-Likelihood:                -7325.6
No. Observations:                1292   AIC:                         1.466e+04
Df Residuals:                    1286   BIC:                         1.469e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                     7.0464     10.905      0.646      0.518     -14.348      28.440
C(Gender2)[T.1]              -3.3154      4.109     -0.807      0.420     -11.377       4.746
C(PrimUse)[T.1]               5.9719      4.385      1.362      0.173      -2.631      14.574
Age                           0.0508      0.154      0.331      0.741      -0.250       0.352
Number_of_Violations         -1.7740      1.062     -1.670      0.095      -3.858       0.310
Estimated_population_2013 -3.709e-05      0.000     -0.243      0.808      -0.000       0.000
==============================================================================
Omnibus:                     3321.060   Durbin-Watson:                   1.974
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         41562442.788
Skew:                         -27.431   Prob(JB):                         0.00
Kurtosis:                     879.954   Cond. No.                     2.41e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.41e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""